In [1]:
# !pip install pytesseract
# !pip install nltk

In [2]:
import pandas as pd
import os
import pytesseract
from PIL import Image
import re

In [3]:
import nltk
nltk.download('words')


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [4]:
# setting parameters
std = 3

if std==3:
    user = "Madhu"
elif std==6:
    user = "bhavi"
else:
    user = "user"

sub = "science"
lesson = 2
file_name = str(std)+"_"+sub+"_LS"+str(lesson)

images_folder_path = 'scanned_files' 
ext_txt_file_path = file_name+'_01_extracted_text.txt'

In [5]:
# list all image files in the folder

def list_files_in_folder(images_folder_path):
    """Lists all files in the given folder."""
    try:
        # List all entries in the given folder
        entries = os.listdir(images_folder_path)
        files = [entry for entry in entries if os.path.isfile(os.path.join(images_folder_path, entry))]
        return files
    except FileNotFoundError:
        print("The folder does not exist.")
        return []
    except PermissionError:
        print("Permission denied.")
        return []

# Example usage
files = list_files_in_folder(images_folder_path)
print("Files in folder:", files)


Files in folder: ['20240302_090846.jpg', '20240302_090904.jpg', '20240302_090937.jpg', '20240302_090953.jpg', '20240302_091024.jpg', '20240302_091043.jpg', '20240302_091121.jpg', '20240302_091141.jpg', '20240302_091211.jpg', '20240302_091229.jpg', '20240302_091430.jpg']


In [6]:
# extract the text from images and save to extracted_text.txt
def extract_text_from_image(image_path):
    # Open the image file
    with Image.open(image_path) as img:
        # Use pytesseract to extract text from the image
        text = pytesseract.image_to_string(img)
        return text

def extract_text_from_list_of_image(files):
    pageno=1
    # Open the file in append mode
    with open(ext_txt_file_path, 'a') as data:
        # Iterate through a range or any iterable
        for file in files:
            image_path = images_folder_path+'/'+file
            page_no = f"\n\nPage nummber: {pageno}"+'\n'
            extracted_text = extract_text_from_image(image_path)
            # Append text to the file
            data.write(page_no+extracted_text)
            pageno+=1

extract_text_from_list_of_image(files)

In [7]:
# get unique words from extracted text
def get_unique_words(ext_txt_file_path):
    unique_words = set()

    # Open the file and read its contents
    with open(ext_txt_file_path, 'r') as file:
        # Read the text and split it into words
        words = file.read().split()

        # Add each word to the set of unique words
        for word in words:
            unique_words.add(word)

    return unique_words

# Example usage:

unique_words = get_unique_words(ext_txt_file_path)
unique_words_list = list(unique_words)
df = pd.DataFrame(unique_words_list)
df.columns=['unique_words']
df.to_csv(file_name+"_02_unique_words.csv",index=False)
df

,unique_words
0,healthy
1,rickets.
2,Carbohydrates
3,list
4,revise
...,...
968,Andhra
969,pan.
970,aranerals)
971,Kitchen


In [8]:
# remove the words that are not in dictionary
import nltk
from nltk.corpus import words

# Download the word list
nltk.download('words')

# List of English words provided by nltk
english_vocab = set(w.lower() for w in words.words())

def filter_english_words(word_list):
    """Filter the given list of words, keeping only English dictionary words."""
    # Convert all words in the list to lower case
    word_list = [word.lower() for word in word_list]
    # Filter the list to keep only words found in the English vocabulary
    return [word for word in word_list if word in english_vocab]

# Example usage:
filtered_words = filter_english_words(unique_words_list)
df_dict_matched_words = pd.DataFrame(filtered_words)
df_dict_matched_words.columns=['dict_matched_words']
df_dict_matched_words.to_csv(file_name+"_03_dict_matched_words.csv", index=False)
df_dict_matched_words


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


,dict_matched_words
0,healthy
1,list
2,revise
3,correct
4,powder
...,...
466,see
467,made
468,we
469,andhra


In [9]:
# Remove unwanted words that are manually notted
df_unwanted_words = pd.read_csv("remove_unwanted_words.csv")
df_unwanted_words_list = df_unwanted_words["unwanted_words"].to_list()

filtered_words = pd.read_csv(file_name+"_03_dict_matched_words.csv")
filtered_words = filtered_words["dict_matched_words"]
filtered_wanted_words = [word for word in filtered_words if word not in df_unwanted_words_list]
df_dict_matched_words = pd.DataFrame(filtered_wanted_words)
df_dict_matched_words.columns=['dict_matched_filtered_words']
df_dict_matched_words.to_csv(file_name+"_04_dict_matched_filtered_words.csv", index=False)
df_dict_matched_words

,dict_matched_filtered_words
0,healthy
1,list
2,revise
3,correct
4,powder
...,...
436,see
437,made
438,we
439,andhra


In [10]:
# chunking
def chunk_around_word(sentence, target_word, sentence_len=10):
    cleaned_text = re.sub(r'\n', ' ', sentence)
    cleaned_text = re.sub(r'[^A-Za-z0-9 ]', '', cleaned_text)
    sentence_lower = cleaned_text.lower()
    target_word = target_word.lower()

    sentence_len_left = int(sentence_len/2)
    
    words = sentence_lower.split()
    target_index = words.index(target_word)  # Find the index of the target word

    # Calculate start and end indices for slicing the words list
    start_index = max(0, target_index - sentence_len_left)  # 5 words before
    end_index = min(len(words), target_index + sentence_len_left)  # 4 words after, +1 because of slice behavior

    # Join and return the chunk
    chunk = ' '.join(words[start_index:end_index])
    return chunk

# find sentance with the given word
def find_sentence_with_word(sentences, word):
    word = word.lower()  # Convert the word to lowercase
    sent = ""
    cleaned_sentence = ""
    for sentence in sentences:
        cleaned_text = re.sub(r'\n', ' ', sentence)
        cleaned_text = re.sub(r'[^A-Za-z0-9 ]', '', cleaned_text)
        if re.search(r'\b{}\b'.format(re.escape(word)), cleaned_text.lower()):
            sent = sentence
            cleaned_sentence = cleaned_text
            break
        else:
            sent = ""
            cleaned_sentence = ""

    return sent, cleaned_sentence
        

In [11]:
# sentance spliter

nltk.download('punkt')  # Download the required tokenizer data

def split_into_sentences(text):
    # Use the nltk tokenizer to split the text into sentences
    sentences = nltk.sent_tokenize(text)
    return sentences

# Read the text file
with open(ext_txt_file_path, "r", encoding="latin-1") as file:
    text = file.read()

# Split the text into sentences
sentences = split_into_sentences(text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# fill the sentence and chunk sentance columns
df = pd.read_csv(file_name+"_04_dict_matched_filtered_words.csv")

# Function to apply to each row
def find_and_chunk(row, sentences):
    word = row['dict_matched_filtered_words']
    sentence, cleaned_sentence = find_sentence_with_word(sentences, word)
    if sentence:
        chunked_sentence = chunk_around_word(sentence, word)
    else:
        chunked_sentence = None
    return pd.Series([sentence, cleaned_sentence, chunked_sentence])

# Apply the function to the DataFrame
df[['sentence', 'cleaned_sentence', 'chunked_sentence']] = df.apply(find_and_chunk, args=(sentences,), axis=1)
df.to_csv(file_name+"_05_words_with_chunked_sentence.csv", index=False)
df


,dict_matched_filtered_words,sentence,cleaned_sentence,chunked_sentence
0,healthy,Food component Food items\n\nProteins\nCarbohy...,Food component Food items Proteins Carbohydra...,to grow well and stay healthy we should have a
1,list, list the harmful effects\nof rotten food.,list the harmful effects of rotten food,list the harmful effects of
2,revise,aed Study Skills Seen\n\nComplete the flow cha...,aed Study Skills Seen Complete the flow chart...,complete the flow chart to revise the differen...
3,correct,Tick (V) the correct option.,Tick V the correct option,tick v the correct option
4,powder,2\nEvnariantial | aa ;\n/ + Experiential Learn...,2 Evnariantial aa Experiential Learning T...,a tablespoon each of coconut powder rice musta...
...,...,...,...,...
436,see,"ame Se eee\na, See\n& wemmoaniaians |\n\neo es...",ame Se eee a See wemmoaniaians eo es rns IV,ame se eee a see wemmoaniaians eo es rns
437,made,=\n\n\n\nPage nummber: 6\nWater\n\nAlong with ...,Page nummber 6 Water Along with food the ...,of the human body is made up of water
438,we,\n\nPage nummber: 1\n2 The Food We Eat\n\nLESS...,Page nummber 1 2 The Food We Eat LESSON SCH...,nummber 1 2 the food we eat lesson schema pss
439,andhra,"Rajasthan \nsaag, makke ki roti Dal-batti-chu...",Rajasthan saag makke ki roti Dalbattichurma ...,saag makke ki roti dalbattichurma andhra prade...


In [13]:
import pandas as pd
import nltk
from nltk.corpus import brown

# Ensure you've downloaded the necessary NLTK resources
nltk.download('brown')
nltk.download('punkt')

# Load all sentences from the Brown Corpus
brown_sentences = [' '.join(sent) for sent in brown.sents()]

def find_example_sentence(word):
    word = word.lower()
    for sentence in brown_sentences:
        if word in sentence.lower().split():
            return sentence
    return "No example sentence found."

def mask_word(word, sentence):
    word = word.lower()
    if re.search(r'\b{}\b'.format(re.escape(word)), sentence.lower()):
        masked_sentence = sentence.lower().replace(word, '*' * len(word))
    else:
        masked_sentence = sentence
    return masked_sentence.capitalize()  # Capitalize the first letter of the sentenc

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eswar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df = pd.read_csv(file_name+"_05_words_with_chunked_sentence.csv")

# Add a column with the masked chunked sentence
df['masked_chunked_sentence'] = df.apply(lambda row: mask_word(row['dict_matched_filtered_words'], row['chunked_sentence']), axis=1)

#adding dummy columns openai_sentence,masked_openai_sentence
df['example_sentence'] = ""
df['masked_example_sentence'] = ""
df['openai_sentence'] = ""
df['masked_openai_sentence'] = ""
df.to_csv(file_name+"_06_words_with_chunked_eg_sentence.csv", index=False)
df.head()


,dict_matched_filtered_words,sentence,cleaned_sentence,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,healthy,Food component Food items\n\nProteins\nCarbohy...,Food component Food items Proteins Carbohydra...,to grow well and stay healthy we should have a,To grow well and stay ******* we should have a,,,,
1,list, list the harmful effects\nof rotten food.,list the harmful effects of rotten food,list the harmful effects of,**** the harmful effects of,,,,
2,revise,aed Study Skills Seen\n\nComplete the flow cha...,aed Study Skills Seen Complete the flow chart...,complete the flow chart to revise the differen...,Complete the flow chart to ****** the differen...,,,,
3,correct,Tick (V) the correct option.,Tick V the correct option,tick v the correct option,Tick v the ******* option,,,,
4,powder,2\nEvnariantial | aa ;\n/ + Experiential Learn...,2 Evnariantial aa Experiential Learning T...,a tablespoon each of coconut powder rice musta...,A tablespoon each of coconut ****** rice musta...,,,,


In [15]:
# update openai sentences from master wordlist
df = pd.read_csv(file_name+"_06_words_with_chunked_eg_sentence.csv")
# Read the word_list_all CSV file into df1
df1 = pd.read_csv('word_list_all.csv')

# Merge df with df1
df_merged = pd.merge(df, df1, how='left', left_on='dict_matched_filtered_words', right_on='word')
df_merged.loc[df_merged['example_sentence_x'].isnull(), "example_sentence_x"]=df_merged.loc[df_merged['example_sentence_x'].isnull(), "example_sentence_y"]
df_merged.loc[df_merged['masked_example_sentence_x'].isnull(), "masked_example_sentence_x"]=df_merged.loc[df_merged['masked_example_sentence_x'].isnull(), "masked_example_sentence_y"]
df_merged.loc[df_merged['openai_sentence_x'].isnull(), "openai_sentence_x"]=df_merged.loc[df_merged['openai_sentence_x'].isnull(), "openai_sentence_y"]
df_merged.loc[df_merged['masked_openai_sentence_x'].isnull(), "masked_openai_sentence_x"]=df_merged.loc[df_merged['masked_openai_sentence_x'].isnull(), "masked_openai_sentence_y"]

df_merged.drop(['user','word','chunked_sentence_y','masked_chunked_sentence_y', 'example_sentence_y', 'masked_example_sentence_y', 'openai_sentence_y', 'masked_openai_sentence_y'], axis=1, inplace=True)
df_merged.rename(columns={'chunked_sentence_x':'chunked_sentence', 'masked_chunked_sentence_x':'masked_chunked_sentence', 'example_sentence_x':'example_sentence', 'masked_example_sentence_x':'masked_example_sentence', 'openai_sentence_x':'openai_sentence', 'masked_openai_sentence_x':'masked_openai_sentence'}, inplace=True)
df_merged['word_type']=file_name
df_merged=df_merged[['dict_matched_filtered_words', 'word_type', 'sentence', 'cleaned_sentence',
       'chunked_sentence', 'masked_chunked_sentence',
       'example_sentence', 'masked_example_sentence',
       'openai_sentence', 'masked_openai_sentence']]
df_merged.drop_duplicates(inplace=True,ignore_index=True)

C:\Users\eswar\AppData\Local\Temp\ipykernel_6712\2671655196.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '["Hirsch's Scapin is healthy , cheerful , energetic , revelling in his physical agility and his obvious superiority to the young gentlemen whom he serves ."
 "Hirsch's Scapin is healthy , cheerful , energetic , revelling in his physical agility and his obvious superiority to the young gentlemen whom he serves ."
 "Hirsch's Scapin is healthy , cheerful , energetic , revelling in his physical agility and his obvious superiority to the young gentlemen whom he serves ."
 ...
 "Nevertheless , `` we feel that in the future Fulton County should receive some portion of these available funds '' , the jurors said ."
 nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_merged.loc[df_merged['example_sentence_x'].isnull(), "example_sentence_x"]=df_merged.l

In [16]:
# Add a column with an example sentence from the Brown Corpus
df_merged.loc[df_merged['example_sentence'].isnull(), 'example_sentence']= df_merged.loc[df_merged['example_sentence'].isnull(), 'dict_matched_filtered_words'].apply(find_example_sentence)

# Add a column with the masked example sentence
df_merged['masked_example_sentence'] = df_merged.apply(lambda row: mask_word(row['dict_matched_filtered_words'], row['example_sentence']), axis=1)
df_merged.to_csv(file_name+"_10_merged_df_with_existing_openai_sentence.csv", index=False)
df_merged.head()

,dict_matched_filtered_words,word_type,sentence,cleaned_sentence,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,healthy,3_science_LS2,Food component Food items\n\nProteins\nCarbohy...,Food component Food items Proteins Carbohydra...,to grow well and stay healthy we should have a,To grow well and stay ******* we should have a,"Hirsch's Scapin is healthy , cheerful , energe...","Hirsch's scapin is ******* , cheerful , energe...",It's important to stay healthy.,It's important to stay *******.
1,list,3_science_LS2, list the harmful effects\nof rotten food.,list the harmful effects of rotten food,list the harmful effects of,**** the harmful effects of,The Central Falls City Council expressed conce...,The central falls city council expressed conce...,Make a list of your toys.,Make a **** of your toys.
2,revise,3_science_LS2,aed Study Skills Seen\n\nComplete the flow cha...,aed Study Skills Seen Complete the flow chart...,complete the flow chart to revise the differen...,Complete the flow chart to ****** the differen...,Tom was not willing to revise the play accordi...,Tom was not willing to ****** the play accordi...,Let's revise our essays.,let's ****** our essays.
3,correct,3_science_LS2,Tick (V) the correct option.,Tick V the correct option,tick v the correct option,Tick v the ******* option,He assured Mr. Martinelli and the council that...,He assured mr. martinelli and the council that...,Circle the correct answer.,Circle the ******* answer.
4,powder,3_science_LS2,2\nEvnariantial | aa ;\n/ + Experiential Learn...,2 Evnariantial aa Experiential Learning T...,a tablespoon each of coconut powder rice musta...,A tablespoon each of coconut ****** rice musta...,"'' Despite Company threats , duly carried thro...","'' despite company threats , duly carried thro...",Add a teaspoon of cocoa powder.,Add a teaspoon of cocoa ******.


In [17]:
# listing null values in openai sentences
df = pd.read_csv(file_name+"_06_words_with_chunked_eg_sentence.csv")
df = df_merged.loc[df_merged['openai_sentence'].isnull(),['dict_matched_filtered_words','openai_sentence']]
df.reset_index(inplace=True, drop=True)
df.to_csv(file_name+"_07_blank_openai_sentence.csv", index=False)
df

,dict_matched_filtered_words,openai_sentence
0,code,NaN
1,prefer,NaN
2,spoil,NaN
3,fa,NaN
4,coastal,NaN
5,wy,NaN
6,vomiting,NaN
7,unique,NaN
8,kitchen,NaN
9,sesame,NaN


In [20]:
# Adding masked openai sentence to new words. 
# Manually enter the openai sentences using chatgpt and then use this funtion to fill masked sentance.
# Add a column with the masked example sentence
try:
    df = pd.read_csv(file_name+"_08_openai_sentence.csv")
    df['masked_openai_sentence'] = df.apply(lambda row: mask_word(row['dict_matched_filtered_words'], row['openai_sentence']), axis=1)
    df.to_csv(file_name+"_09_masked_openai_sentence.csv", index=False)
except:
    print("Manually enter the openai sentences using chatgpt and then use this funtion to fill masked sentance.")

df.head()

,dict_matched_filtered_words,openai_sentence,masked_openai_sentence
0,aide,The teacher's aide helps us.,the teacher's **** helps us.
1,andhra,Andhra Pradesh is in India.,****** pradesh is in india.
2,athlete,She is a talented athlete.,she is a talented *******.
3,bitter,The medicine tasted bitter.,the medicine tasted ******.
4,bone,The dog chewed on a bone.,the dog chewed on a ****.


In [21]:
# merging the new openai sentances with existing sheet
df =  pd.read_csv(file_name+"_10_merged_df_with_existing_openai_sentence.csv")
df1 = pd.read_csv(file_name+"_09_masked_openai_sentence.csv")
df1.columns = ['dict_matched_filtered_words', 'openai_sentence_new', 'masked_openai_sentence_new']

df_merged = pd.merge(df, df1, how='left', on='dict_matched_filtered_words')
df_merged.loc[df_merged['openai_sentence'].isnull(), "openai_sentence"]=df_merged.loc[df_merged['openai_sentence'].isnull(), "openai_sentence_new"]
df_merged.loc[df_merged['masked_openai_sentence'].isnull(), "masked_openai_sentence"]=df_merged.loc[df_merged['masked_openai_sentence'].isnull(), "masked_openai_sentence_new"]
df_merged.drop(columns=['sentence', 'cleaned_sentence', 'openai_sentence_new','masked_openai_sentence_new'], inplace=True)
df_merged.rename(columns={'dict_matched_filtered_words':'word'}, inplace=True)

df_merged.to_csv(file_name+"_11_word_list.csv", index=False)
df_merged

,word,word_type,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,healthy,3_science_LS2,to grow well and stay healthy we should have a,To grow well and stay ******* we should have a,"Hirsch's Scapin is healthy , cheerful , energe...","Hirsch's scapin is ******* , cheerful , energe...",It's important to stay healthy.,It's important to stay *******.
1,list,3_science_LS2,list the harmful effects of,**** the harmful effects of,The Central Falls City Council expressed conce...,The central falls city council expressed conce...,Make a list of your toys.,Make a **** of your toys.
2,revise,3_science_LS2,complete the flow chart to revise the differen...,Complete the flow chart to ****** the differen...,Tom was not willing to revise the play accordi...,Tom was not willing to ****** the play accordi...,Let's revise our essays.,let's ****** our essays.
3,correct,3_science_LS2,tick v the correct option,Tick v the ******* option,He assured Mr. Martinelli and the council that...,He assured mr. martinelli and the council that...,Circle the correct answer.,Circle the ******* answer.
4,powder,3_science_LS2,a tablespoon each of coconut powder rice musta...,A tablespoon each of coconut ****** rice musta...,"'' Despite Company threats , duly carried thro...","'' despite company threats , duly carried thro...",Add a teaspoon of cocoa powder.,Add a teaspoon of cocoa ******.
...,...,...,...,...,...,...,...,...
387,human,3_science_LS2,the human body needs activities,The ***** body needs activities,It made him human .,It made him ***** .,Every human has a heart.,Every ***** has a heart.
388,elderly,3_science_LS2,age elderly people may like to,Age ******* people may like to,Of the two cherished achievements the elderly ...,Of the two cherished achievements the ******* ...,The elderly man needs assistance.,the ******* man needs assistance.
389,see,3_science_LS2,ame se eee a see wemmoaniaians eo es rns,Ame se eee a *** wemmoaniaians eo es rns,"Barber , who is in his 13th year as a legislat...","Barber , who is in his 13th year as a legislat...",I see a bird in the tree.,I *** a bird in the tree.
390,made,3_science_LS2,of the human body is made up of water,Of the human body is **** up of water,"Before adjournment Monday afternoon , the Sena...","Before adjournment monday afternoon , the sena...",I made a paper airplane.,I **** a paper airplane.


In [22]:
#updating the word list and word list all
df1 = pd.read_csv("word_list.csv")
df2 =  pd.read_csv(file_name+"_11_word_list.csv")
df2['user'] = user
columns=['user', 'word', 'word_type', 'chunked_sentence', 'masked_chunked_sentence', 'example_sentence', 'masked_example_sentence', 'openai_sentence', 'masked_openai_sentence']
df2=df2[columns]
df_appended = pd.concat([df1, df2])
df_appended = df_appended.drop_duplicates()
df_appended.to_csv("word_list.csv", index=False)
df_appended


,user,word,word_type,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,user,about,Sight Words,NaN,NaN,The Highway Department source told The Constit...,The highway department source told the constit...,This book is about animals.,This book is ***** animals.
1,user,after,Sight Words,NaN,NaN,"He will be succeeded by Ivan Allen Jr. , who b...","He will be succeeded by ivan allen jr. , who b...",Lunch is after class.,Lunch is ***** class.
2,user,all,Sight Words,NaN,NaN,`` This is one of the major items in the Fulto...,`` this is one of the major items in the fulto...,All the cookies are gone.,*** the cookies are gone.
3,user,am,Sight Words,NaN,NaN,`` But I am not in favor of a sales or state i...,`` but i ** not in favor of a sales or state i...,I am happy today.,I ** happy today.
4,user,an,Sight Words,NaN,NaN,The Fulton County Grand Jury said Friday an in...,The fulton county gr**d jury said friday ** in...,I saw an elephant.,I saw ** eleph**t.
...,...,...,...,...,...,...,...,...,...
387,Madhu,human,3_science_LS2,the human body needs activities,The ***** body needs activities,It made him human .,It made him ***** .,Every human has a heart.,Every ***** has a heart.
388,Madhu,elderly,3_science_LS2,age elderly people may like to,Age ******* people may like to,Of the two cherished achievements the elderly ...,Of the two cherished achievements the ******* ...,The elderly man needs assistance.,the ******* man needs assistance.
389,Madhu,see,3_science_LS2,ame se eee a see wemmoaniaians eo es rns,Ame se eee a *** wemmoaniaians eo es rns,"Barber , who is in his 13th year as a legislat...","Barber , who is in his 13th year as a legislat...",I see a bird in the tree.,I *** a bird in the tree.
390,Madhu,made,3_science_LS2,of the human body is made up of water,Of the human body is **** up of water,"Before adjournment Monday afternoon , the Sena...","Before adjournment monday afternoon , the sena...",I made a paper airplane.,I **** a paper airplane.


In [23]:
#updating the word list and word list all
df1 = pd.read_csv("word_list_all.csv")
df2 =  pd.read_csv(file_name+"_11_word_list.csv")
df2['user'] = user
columns=['user', 'word', 'word_type', 'chunked_sentence', 'masked_chunked_sentence', 'example_sentence', 'masked_example_sentence', 'openai_sentence', 'masked_openai_sentence']
df2=df2[columns]
df_appended = pd.concat([df1, df2])
df_appended = df_appended.drop_duplicates()
df_appended.to_csv("word_list_all.csv", index=False)
df_appended

,user,word,word_type,chunked_sentence,masked_chunked_sentence,example_sentence,masked_example_sentence,openai_sentence,masked_openai_sentence
0,user,about,Sight Words,NaN,NaN,The Highway Department source told The Constit...,The highway department source told the constit...,This book is about animals.,This book is ***** animals.
1,user,after,Sight Words,NaN,NaN,"He will be succeeded by Ivan Allen Jr. , who b...","He will be succeeded by ivan allen jr. , who b...",Lunch is after class.,Lunch is ***** class.
2,user,all,Sight Words,NaN,NaN,`` This is one of the major items in the Fulto...,`` this is one of the major items in the fulto...,All the cookies are gone.,*** the cookies are gone.
3,user,am,Sight Words,NaN,NaN,`` But I am not in favor of a sales or state i...,`` but i ** not in favor of a sales or state i...,I am happy today.,I ** happy today.
4,user,an,Sight Words,NaN,NaN,The Fulton County Grand Jury said Friday an in...,The fulton county gr**d jury said friday ** in...,I saw an elephant.,I saw ** eleph**t.
...,...,...,...,...,...,...,...,...,...
387,Madhu,human,3_science_LS2,the human body needs activities,The ***** body needs activities,It made him human .,It made him ***** .,Every human has a heart.,Every ***** has a heart.
388,Madhu,elderly,3_science_LS2,age elderly people may like to,Age ******* people may like to,Of the two cherished achievements the elderly ...,Of the two cherished achievements the ******* ...,The elderly man needs assistance.,the ******* man needs assistance.
389,Madhu,see,3_science_LS2,ame se eee a see wemmoaniaians eo es rns,Ame se eee a *** wemmoaniaians eo es rns,"Barber , who is in his 13th year as a legislat...","Barber , who is in his 13th year as a legislat...",I see a bird in the tree.,I *** a bird in the tree.
390,Madhu,made,3_science_LS2,of the human body is made up of water,Of the human body is **** up of water,"Before adjournment Monday afternoon , the Sena...","Before adjournment monday afternoon , the sena...",I made a paper airplane.,I **** a paper airplane.
